# SRF1

The data in this notebook were dervied from [Bogaert E, et al. (2023) SRSF1 haploinsufficiency is responsible for a syndromic developmental disorder associated with intellectual disability. Am J Hum Genet; 110(5):790-808. PMID: 37071997]

Individual 6 and 17 were removed from the cohort because no SRF1 variant could be indentified in the original study.

In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', None) # show entire column contents, important!
from IPython.display import display, HTML
from pyphetools.creation import *
from pyphetools.visualization import *
from pyphetools.validation import *
import pyphetools
print(f"Using pyphetools version {pyphetools.__version__}")

Using pyphetools version 0.9.66


In [2]:
PMID = "PMID:37071997"
title = "SRSF1 haploinsufficiency is responsible for a syndromic developmental disorder associated with intellectual disability"
cite = Citation(pmid=PMID, title=title)
parser = HpoParser(hpo_json_file="../hp.json")
hpo_cr = parser.get_hpo_concept_recognizer()
hpo_version = parser.get_version()
hpo_ontology = parser.get_ontology()
metadata = MetaData(created_by="ORCID:0000-0002-5648-2155", citation=cite)
metadata.default_versions_with_hpo(version=hpo_version)
print(f"HPO version {hpo_version}")

HPO version 2024-03-06


In [3]:
df = pd.read_csv("input/suppdata.csv")
dft = df.transpose()
dft.columns = dft.iloc[0] 
dft.drop(dft.index[0], inplace=True)
dft['individual_id'] = dft.index 
dft.tail(2)

Subject           Sex SRSF1 Genomic coordinates [ hg19 )  \
Individual 15    Male       NC_000017.10 : 56083832A > C   
Individual 16  Female   NC 000017.10 : g . 56084369C > T   

Subject       Transcript ( NM_006924.5 )           Protein  \
Individual 15                 c.251T > G  p . ( Leu84Arg )   
Individual 16                 c.130G > A  p . ( Asp44Asn )   

Subject       Mode of inheritance  \
Individual 15             De novo   
Individual 16             De novo   

Subject                                               Pregnancy complications  \
Individual 15  Screening test positive : amniocentesis with karyotype 46 , XY   
Individual 16                                                              No   

Subject       Gestational weeks( weeks + days ) Birth weight g ( SD )  \
Individual 15                                39           2980 ( -1 )   
Individual 16                            38 + 6         3065 ( +0.1 )   

Subject       Birth length cm ( SD ) Birth OFC cm ( SD )  ...  \
Individual 15               50 ( 0 )         35 ( +0.2 )  ...   
Individual 16            51 ( +1.3 )         34 ( +0.2 )  ...   

Subject       Urogenital / kidney malformations Other abdominal Scoliosis  \
Individual 15       Balanopreputial hypospadias               -       Yes   
Individual 16                                No             NaN        No   

Subject        Pectus deformity  \
Individual 15  Pectus carinatum   
Individual 16                No   

Subject                                                            Other physical  \
Individual 15  Marfanoid habitus with arachnodactyly Genu valgum Metatarsus varus   
Individual 16                                                                 NaN   

Subject                                                                                                                                            Facial features  \
Individual 15  Triangular face; with facial asymmetry; Upslanted palpebral fissures; Long nose; and tubular nose; Long philtrum; and smooth philtrum; Retrognathia   
Individual 16                                                        Downslanted palpebral fissures; Depressed nasal bridge; Midly downturned corners of the mouth   

Subject       Cutaneous abnormalities Additional clinical features  \
Individual 15                      No                  Bifid uvula   
Individual 16                      No                           No   

Subject       Additional molecular features  individual_id  
Individual 15                           NaN  Individual 15  
Individual 16                           NaN  Individual 16  

[2 rows x 43 columns]

In [4]:
generator = SimpleColumnMapperGenerator(df=dft,
                                  observed='Yes',
                                  excluded='No',
                                  hpo_cr=hpo_cr)
column_mapper_list = generator.try_mapping_columns()
display(HTML(generator.to_html()))

In [5]:
#output = OptionColumnMapper.autoformat(df=dft, hpo_cr=hpo_cr, delimiter=";")
#print(output)

In [6]:
complications_d = {'Oxygen requirement because of pulmonary hypoplasia': 'Pulmonary hypoplasia',
 #'No': 'PLACEHOLDER',
 'Congenital torticoli': 'Congenital muscular torticollis',
 #'RCIU': 'PLACEHOLDER',
 'hypocalcemia': 'Hypocalcemia',
 'Feeding difficulties Icterus': ['Feeding difficulties', 'Jaundice'],
 'Congenital heart defect': 'Abnormal heart morphology',
 'Transient tachypnea Increased muscle tone': ['Tachypnea' 'Hypertonia'],
 'Icterus': 'Jaundice',
 'Neonatal hypotonia Sucking difficulties': ['Neonatal hypotonia','Poor suck'],
 'Transient tachypnea of the newborn': 'Tachypnea'}
complicationsMapper = OptionColumnMapper(column_name='Neonatal complications',concept_recognizer=hpo_cr, option_d=complications_d)
column_mapper_list.append(complicationsMapper)
complicationsMapper.preview_column(dft)

mapping count
0             Pulmonary hypoplasia (HP:0002089) (observed)     1
1  Congenital muscular torticollis (HP:0005988) (observed)     1
2                     Hypocalcemia (HP:0002901) (observed)     1
3                        Tachypnea (HP:0002789) (observed)     2
4                       Hypertonia (HP:0001276) (observed)     1
5                         Jaundice (HP:0000952) (observed)     2
6               Neonatal hypotonia (HP:0001319) (observed)     1

In [7]:
pregnancy_complications_d = {'IUGR': 'Intrauterine growth retardation',
 'Poor active fetal movements': 'Decreased fetal movement',
 'Ovarian stimulation C - section for breech positioning': 'Breech presentation',
 'Premature rupture of membranes at 31w + 2d': 'Premature rupture of membranes',
 'C - section for fetal distress': 'Fetal distress',
 # 'Toxemia C - section': 'Toxemia of pregnancy', 
 'Screening test positive : amniocentesis with karyotype 46': 'Female external genitalia in individual with 46,XY karyotype',
 'Increased nuchal tranlucency': 'Increased nuchal translucency'}
pregnancy_complicationsMapper = OptionColumnMapper(column_name='Pregnancy complications',
                                                   concept_recognizer=hpo_cr, option_d=pregnancy_complications_d)
column_mapper_list.append(pregnancy_complicationsMapper)
pregnancy_complicationsMapper.preview_column(dft)

mapping  \
0                               Intrauterine growth retardation (HP:0001511) (observed)   
1                                      Decreased fetal movement (HP:0001558) (observed)   
2                                           Breech presentation (HP:0001623) (observed)   
3                                Premature rupture of membranes (HP:0001788) (observed)   
4                                                Fetal distress (HP:0025116) (observed)   
5  Female external genitalia in individual with 46,XY karyotype (HP:0008730) (observed)   

  count  
0     1  
1     1  
2     1  
3     1  
4     1  
5     1

In [8]:
gestational_weeks_d = {'36': 'Premature birth', '31 + 2': 'Premature birth'}
gestational_weeksMapper = OptionColumnMapper(column_name='Gestational weeks( weeks + days )',
                                             concept_recognizer=hpo_cr, option_d=gestational_weeks_d)
column_mapper_list.append(gestational_weeksMapper)
gestational_weeksMapper.preview_column(dft)

mapping count
0  Premature birth (HP:0001622) (observed)     2

In [9]:
birth_weight_g_d = {'1700 ( -2.6 )': 'Small for gestational age',
 '1765 ( +0.6 )': 'Small for gestational age' }
birth_weight_g_Mapper = OptionColumnMapper(column_name='Birth weight g ( SD )',concept_recognizer=hpo_cr, option_d=birth_weight_g_d)
column_mapper_list.append(birth_weight_g_Mapper)
birth_weight_g_Mapper.preview_column(dft)

mapping count
0  Small for gestational age (HP:0001518) (observed)     2

In [10]:
birth_length_cm_d = {'39 ( -3.6 )': 'Birth length less than 3rd percentile',
 '40.5 ( -0.2 )': 'Birth length less than 3rd percentile',
 '45 ( -1.8 )': 'Birth length less than 3rd percentile',
 '34 ( -0.5 )': 'Birth length less than 3rd percentile',
}
birth_length_cm_Mapper = OptionColumnMapper(column_name='Birth length cm ( SD )',concept_recognizer=hpo_cr, option_d=birth_length_cm_d)
column_mapper_list.append(birth_length_cm_Mapper)
birth_length_cm_Mapper.preview_column(dft)

mapping count
0  Birth length less than 3rd percentile (HP:0003561) (observed)     4

In [11]:
failure_to_thrive_d = {'Yes': 'Failure to thrive',
 'Yes Chewing diffuculty': ['Failure to thrive', 'Impaired mastication'],
 'Infant reflux': 'Gastroesophageal reflux'}
excluded = {'No': 'Failure to thrive', } 
failure_to_thriveMapper = OptionColumnMapper(column_name='Failure to thrive',
                                             concept_recognizer=hpo_cr, option_d=failure_to_thrive_d, excluded_d=excluded)
column_mapper_list.append(failure_to_thriveMapper)
failure_to_thriveMapper.preview_column(dft)

mapping count
0        Failure to thrive (HP:0001508) (observed)     4
1        Failure to thrive (HP:0001508) (excluded)    10
2     Impaired mastication (HP:0005216) (observed)     1
3  Gastroesophageal reflux (HP:0002020) (observed)     1

In [12]:
truncal_overweight_d = {'Yes': 'Childhood-onset truncal obesity',
 'Trunk adiposity': 'Childhood-onset truncal obesity',
 'Mildly overweight': 'Overweight'}
excluded = {'No': 'Childhood-onset truncal obesity',}
truncal_overweightMapper = OptionColumnMapper(column_name='Truncal overweight',
                                              concept_recognizer=hpo_cr, option_d=truncal_overweight_d, excluded_d=excluded)
column_mapper_list.append(truncal_overweightMapper)
truncal_overweightMapper.preview_column(dft)

mapping count
0  Childhood-onset truncal obesity (HP:0008915) (observed)     2
1  Childhood-onset truncal obesity (HP:0008915) (excluded)    12
2                       Overweight (HP:0025502) (observed)     1

In [13]:
id_d = {'Yes': ['Intellectual disability', 'Neurodevelopmental delay'],
 'Learning difficulties': 'Neurodevelopmental abnormality'}
idMapper = OptionColumnMapper(column_name='Intellectual Disability / Developmental Delay',
                              concept_recognizer=hpo_cr, option_d=id_d)
column_mapper_list.append(idMapper)
idMapper.preview_column(dft)

mapping count
0         Intellectual disability (HP:0001249) (observed)    14
1        Neurodevelopmental delay (HP:0012758) (observed)    14
2  Neurodevelopmental abnormality (HP:0012759) (observed)     1

In [14]:
level_of_id_dd_d = {'Mild to moderate': 'Intellectual disability, moderate',
 'Mild to moderale': 'Intellectual disability, moderate',
 'IQ : 76 Verbal Comprehension Index : 92 Perceptual Reasoning Index : 84 Working memory Index : 66 Processing Speed Index : 75': 'Intellectual disability, borderline',
 'Mild': 'Intellectual disability, mild',
 'Severe ID Age Equivalent Scores at 5 years and 2months : developmental age : 6 months receptive language : 9 months expressive language : 5 months fine motor : 1 year 7 months interpersonal relationship : 1 year 6 months': 'Intellectual disability, severe',
 'Severe': 'Intellectual disability, severe',
 'Borderline level': 'Intellectual disability, borderline',
 'Moderate': 'Intellectual disability, moderate',
 'Cognitive evaluation ( Bayley II test ) at 3 years and 4 months : developmental age estimated at29 months old': 'Intellectual disability, mild'}
level_of_id_ddMapper = OptionColumnMapper(column_name='Level of ID / DD',
                                          concept_recognizer=hpo_cr, option_d=level_of_id_dd_d)
column_mapper_list.append(level_of_id_ddMapper)
level_of_id_ddMapper.preview_column(dft)

mapping count
0    Intellectual disability, moderate (HP:0002342) (observed)     4
1  Intellectual disability, borderline (HP:0006889) (observed)     2
2        Intellectual disability, mild (HP:0001256) (observed)     4
3      Intellectual disability, severe (HP:0010864) (observed)     3

In [15]:
first_words_d = {
 #'18 months': 'PLACEHOLDER',
 'After 18 months': 'Delayed speech and language development',
 '15 years': 'Delayed speech and language development', 
 '24 months': 'Delayed speech and language development',
 '> 24 months': 'Delayed speech and language development'}
first_wordsMapper = OptionColumnMapper(column_name='First words',
                                       concept_recognizer=hpo_cr, option_d=first_words_d)
column_mapper_list.append(first_wordsMapper)
first_wordsMapper.preview_column(dft)

mapping count
0  Delayed speech and language development (HP:0000750) (observed)     6

In [16]:
speech_d = {'Short sentences': 'Delayed speech and language development',
 '50-100 words - poor articulation': 'Speech articulation difficulties',
 'Speech delay': 'Delayed speech and language development',
 'Delayed ( simple sentences )': 'Delayed speech and language development',
 'No speech': 'Absent speech',
 'mild articulation issue': 'Speech articulation difficulties',
 '2 words': 'Delayed speech and language development',
 'Simplified sentences': 'Delayed speech and language development',
 'Dealyed': 'Delayed speech and language development',
 'few words ( one syllable )': 'Delayed speech and language development',
 'Delayed - 30 single words at 2 years 2 months': 'Delayed speech and language development',
 'Poor ( 10-15 spontaneous words )': 'Delayed speech and language development'}
speechMapper = OptionColumnMapper(column_name='Speech at last examination', concept_recognizer=hpo_cr, option_d=speech_d)
column_mapper_list.append(speechMapper)
speechMapper.preview_column(dft)

mapping count
0  Delayed speech and language development (HP:0000750) (observed)     9
1         Speech articulation difficulties (HP:0009088) (observed)     2
2                            Absent speech (HP:0001344) (observed)     2

In [17]:
sitting_d = {
 '12 months': 'Delayed ability to sit',
 '10 months': 'Delayed ability to sit',
 '10 years': 'Delayed ability to sit',
 '11 months': 'Delayed gross motor development'}
sittingMapper = OptionColumnMapper(column_name='Sitting position age',concept_recognizer=hpo_cr, option_d=sitting_d)
column_mapper_list.append(sittingMapper)
sittingMapper.preview_column(dft)

mapping count
0           Delayed ability to sit (HP:0025336) (observed)     5
1  Delayed gross motor development (HP:0002194) (observed)     1

In [18]:
# Threshold of 18 months
walking_age_d = {'4 years': 'Delayed ability to walk',
 '29 months': 'Delayed ability to walk',
 'Not acquired at last visit Up unaided : 21monts': 'Delayed ability to walk',
 '19 months': 'Delayed ability to walk',
 '23 months': 'Delayed ability to walk',
 '2 years 4 months': 'Delayed ability to walk',
 '2years 6 months': 'Delayed ability to walk',
 '11 years': 'Delayed ability to walk',
 '20 months': 'Delayed ability to walk',
 '1 year 11 months': 'Delayed ability to walk',
 '18-19 months': 'Delayed ability to walk',
 '24 months': 'Delayed ability to walk'}
walking_ageMapper = OptionColumnMapper(column_name='Walking age',concept_recognizer=hpo_cr, option_d=walking_age_d)
column_mapper_list.append(walking_ageMapper)
walking_ageMapper.preview_column(dft)

mapping count
0  Delayed ability to walk (HP:0031936) (observed)    11

In [19]:
hypotonia_d = {'Yes': 'Hypotonia',
 'Yes during infancy': 'Infantile muscular hypotonia',
 'Buccal hypotonia': 'Facial hypotonia',
 }
excluded = { 'No': 'Hypotonia',}
hypotoniaMapper = OptionColumnMapper(column_name='Hypotonia',concept_recognizer=hpo_cr, option_d=hypotonia_d, excluded_d=excluded)
column_mapper_list.append(hypotoniaMapper)
hypotoniaMapper.preview_column(dft)

mapping count
0                     Hypotonia (HP:0001252) (observed)     9
1  Infantile muscular hypotonia (HP:0008947) (observed)     1
2              Facial hypotonia (HP:0000297) (observed)     1
3                     Hypotonia (HP:0001252) (excluded)     3

In [20]:
autism_d = { 
 'Mild at 4 years with difficulties to adaptation to new situations': 'Reduced social reciprocity',
 'frustration intolerance': 'Low frustration tolerance',
 'rare heteroagressivity': 'Aggression towards others',
 #'resolved at 6 years': 'PLACEHOLDER',
 'Repetitive motions': 'Abnormal repetitive mannerisms',
 'very poor social interaction': 'Reduced social reciprocity',
 'overreaction to frustration': 'Low frustration tolerance',
 'Autistic featues but no formal ASD diagnosis Sensory issues better with age': ['Autistic behavior', 'Sensory behavioral abnormality'],
 'Flapping of the hands Feeling overwhelmed in social situations\nFeeling overwhelmed in social situations .': ['Recurrent hand flapping','Abnormal social behavior'], 
 'Social difficulties Obsessions Stereotypies': ['Insistence on sameness', 'Abnormal repetitive mannerisms'],
 'Stereotypies': 'Abnormal repetitive mannerisms',
 'Poor eye contact': 'Reduced eye contact',
 'No response to her name': 'Abnormal response to social norms',
 'Afraid of loud noises': 'Phonophobia',
 'Lines things up in a row': 'Atypical sorting',
 'Autistic features': 'Autistic behavior'}
excluded = {'No': 'Autistic behavior'}
asdMapper = OptionColumnMapper(column_name='Autistic features ( please describe )',
                               concept_recognizer=hpo_cr, option_d=autism_d, excluded_d=excluded)
column_mapper_list.append(asdMapper)
asdMapper.preview_column(dft)

mapping count
0                   Autistic behavior (HP:0000729) (excluded)     8
1          Reduced social reciprocity (HP:0012760) (observed)     2
2           Low frustration tolerance (HP:0000744) (observed)     2
3           Aggression towards others (HP:5200125) (observed)     1
4      Abnormal repetitive mannerisms (HP:0000733) (observed)     3
5                   Autistic behavior (HP:0000729) (observed)     1
6      Sensory behavioral abnormality (HP:5200046) (observed)     1
7              Insistence on sameness (HP:5200130) (observed)     1
8                 Reduced eye contact (HP:0000817) (observed)     1
9   Abnormal response to social norms (HP:5200123) (observed)     1
10                        Phonophobia (HP:0002183) (observed)     1
11                   Atypical sorting (HP:5200049) (observed)     1

In [21]:
outbursts_of_anger_d = {'Yes': 'Abnormal temper tantrums',
 'Few': 'Abnormal temper tantrums',
 'until 9months': 'Abnormal temper tantrums',
 'less frequent with age': 'Abnormal temper tantrums',
 'Yes with frustration or specific circumstances': 'Abnormal temper tantrums'}
excluded = {'No': 'Abnormal temper tantrums'}
outbursts_of_angerMapper = OptionColumnMapper(column_name='Outbursts of anger',
                                              concept_recognizer=hpo_cr, option_d=outbursts_of_anger_d, excluded_d=excluded)
column_mapper_list.append(outbursts_of_angerMapper)
outbursts_of_angerMapper.preview_column(dft)

mapping count
0  Abnormal temper tantrums (HP:0025160) (observed)    11
1  Abnormal temper tantrums (HP:0025160) (excluded)     6

In [22]:
other_autistic_d = { #'nan': 'PLACEHOLDER',
 'Stereotypy': 'Abnormal repetitive mannerisms',
 'Happy': 'Happy demeanor',
 'smiling': 'Happy demeanor',
 'ungoing': 'Social disinhibition',
 'Outbursts of crying or laughter': 'Abnormal affect',
 'ADHD diagnosed at 5 years': 'Attention deficit hyperactivity disorder',
 'Hyperactivity Tiptoe walking': 'Tip-toe gait',
  'Intentional tremor': 'Tremor', 
 'Generalized anxiety disorder with phobic obsessive features': ['Anxiety', 'Phobia']}
other_autistic_Mapper = OptionColumnMapper(column_name='Other ASD features',concept_recognizer=hpo_cr, option_d=other_autistic_d)
column_mapper_list.append(other_autistic_Mapper)
other_autistic_Mapper.preview_column(dft)

mapping count
0            Abnormal repetitive mannerisms (HP:0000733) (observed)     1
1                            Happy demeanor (HP:0040082) (observed)     2
2                      Social disinhibition (HP:5200029) (observed)     1
3                           Abnormal affect (HP:5200261) (observed)     1
4  Attention deficit hyperactivity disorder (HP:0007018) (observed)     1
5                              Tip-toe gait (HP:0030051) (observed)     1
6                                    Tremor (HP:0001337) (observed)     1
7                                   Anxiety (HP:0000739) (observed)     1
8                                    Phobia (HP:5200232) (observed)     1

In [23]:
hearing_loss_d = { 
 'Bilateral low frequency hearing loss': 'Low-frequency hearing loss'}
excluded = {'No': 'Hearing impariment',}
hearing_lossMapper = OptionColumnMapper(column_name='Hearing loss',concept_recognizer=hpo_cr, option_d=hearing_loss_d)
column_mapper_list.append(hearing_lossMapper)
hearing_lossMapper.preview_column(dft)

mapping count
0  Low-frequency hearing loss (HP:0008542) (observed)     1

In [24]:
vision_problems_d = { 'Strabismus': 'Strabismus',
 'myopia': 'Myopia',
 'Myopia': 'Myopia',
 'Astigmatism and Strabismus': ['Strabismus', 'Astigmatism'],
 'Astigmatism Hypermetropia': ['Hypermetropia', 'Astigmatism'],
 'Hypermetropia': 'Hypermetropia',
 'Nystagmus Strabismus': ['Strabismus', 'Nystagmus'],
 'Severe unilateral myopia': 'Myopia'}
excluded = {'No': 'Abnormality of vision',}
vision_problemsMapper = OptionColumnMapper(column_name='Vision problems',
                                           concept_recognizer=hpo_cr, option_d=vision_problems_d, excluded_d=excluded)
column_mapper_list.append(vision_problemsMapper)
vision_problemsMapper.preview_column(dft)

mapping count
0             Strabismus (HP:0000486) (observed)     3
1                 Myopia (HP:0000545) (observed)     5
2            Astigmatism (HP:0000483) (observed)     1
3  Abnormality of vision (HP:0000504) (excluded)     7
4          Hypermetropia (HP:0000540) (observed)     1
5              Nystagmus (HP:0000639) (observed)     1

In [25]:
seizures_d = { 
 'EEG : some anomalies during the process of falling asleep': 'EEG abnormality',
 'Febrile seizure ( one time in in fancy )': 'Febrile seizure (within the age range of 3 months to 6 years)',
 'Myoclonus with fever': 'Myoclonus',
 'Staring spells with deviant gaze': 'Staring gaze'}
excluded = {'No': 'Seizure',
            'No seizure': 'Seizure',
            'No seizure Normal EEG': 'Seizure',
            'No seizure,Normal EEG': 'Seizure',
            'Normal EEG': 'Seizure',
            'No clinical seizure reported': 'Seizure',
            'not associated with clinical signs': 'Seizure',}
seizuresMapper = OptionColumnMapper(column_name='Seizures',concept_recognizer=hpo_cr, option_d=seizures_d, excluded_d=excluded)
column_mapper_list.append(seizuresMapper)
seizuresMapper.preview_column(dft)

mapping  \
0                                                        Seizure (HP:0001250) (excluded)   
1                                                        Seizure (HP:0001250) (observed)   
2                                                EEG abnormality (HP:0002353) (observed)   
3  Febrile seizure (within the age range of 3 months to 6 years) (HP:0002373) (observed)   
4                                                      Myoclonus (HP:0001336) (observed)   
5                                                   Staring gaze (HP:0025401) (observed)   

  count  
0    11  
1     1  
2     1  
3     1  
4     1  
5     1

In [26]:
brain_mri_d = {'Global widening of the ventricular system': 'Ventriculomegaly',
 'cortical subcortical atrophy': 'Subcortical cerebral atrophy',
 'low amount of myelin': 'CNS hypomyelination',
'Myelinisation delay': 'Delayed CNS myelination',
 'Thin splenium of the corpus callosum Leptomeningeal cyst of the left temporal lobe': 'Thin corpus callosum',
 'Microcephaly Small frontal lobes with abnormal white matter intensity Simplification of gyral pattern Delayed subcortical white matter myelination Small brain stem Atrophic lateral rectus muscles': ['Microcephaly', 'Hypoplasia of the frontal lobes', 'Hyperintensity of cerebral white matter on MRI', 'Simplified gyral pattern', 'CNS demyelination', 'Aplasia/Hypoplasia of the brainstem', 'Lateral rectus muscle weakness'],
 'Old bilateral caudothalamic groove (germinal matrix)hemorrhages and few punctate foci of old hemorrhage in the cerebellum. Few scattered foci of periventricular and subcortical T2 hyper intensity and Nonspecific but may be sequelae of prior injury': ['Grade I preterm intraventricular hemorrhage', 'Punctate periventricular T2 hyperintense foci'],
 'Midline pituitary cyst which may represent pars intermedia cyst or Rathes cleft cyst Otherwise normal': 'Pituitary gland cyst'
                                        }
brain_mriMapper = OptionColumnMapper(column_name='Brain Structural anomalies from MRI',concept_recognizer=hpo_cr, option_d=brain_mri_d)
column_mapper_list.append(brain_mriMapper)
brain_mriMapper.preview_column(dft)

mapping  \
0                                 Ventriculomegaly (HP:0002119) (observed)   
1                     Subcortical cerebral atrophy (HP:0012157) (observed)   
2                              CNS hypomyelination (HP:0003429) (observed)   
3                          Delayed CNS myelination (HP:0002188) (observed)   
4                             Thin corpus callosum (HP:0033725) (observed)   
5                                     Microcephaly (HP:0000252) (observed)   
6                  Hypoplasia of the frontal lobes (HP:0007333) (observed)   
7   Hyperintensity of cerebral white matter on MRI (HP:0030890) (observed)   
8                         Simplified gyral pattern (HP:0009879) (observed)   
9                                CNS demyelination (HP:0007305) (observed)   
10             Aplasia/Hypoplasia of the brainstem (HP:0007362) (observed)   
11                  Lateral rectus muscle weakness (HP:0031750) (observed)   

   count  
0      1  
1      1  
2      1  
3      1  
4      1  
5      1  
6      1  
7      1  
8      1  
9      1  
10     1  
11     1

In [27]:
cardiac_malformations_d = {'ASD Pericardial effusion': 'Pericardial effusion',
 'Small ovale foramen': 'Patent foramen ovale',
# 'No': 'PLACEHOLDER',
 'Tricupsid valve leakage': 'Tricuspid valve regurgitation',
 'Ventricular septal defect ( spontaneous closure ) Right aortic arch Aberrant subclavian artery with subclavian steal': ['Ventricular septal defect', 'Right aortic arch','Abnormal subclavian artery morphology'],
 'Minimal aortic regurgitation Mitral regurgitation': 'Aortic regurgitation',
 'Myxoid dystrophy Cardiomegaly Mitral insufficiency': 'Mitral regurgitation'}
cardiac_malformationsMapper = OptionColumnMapper(column_name='Cardiac malformations',concept_recognizer=hpo_cr, option_d=cardiac_malformations_d)
column_mapper_list.append(cardiac_malformationsMapper)
cardiac_malformationsMapper.preview_column(dft)

mapping count
0                   Pericardial effusion (HP:0001698) (observed)     1
1                   Patent foramen ovale (HP:0001655) (observed)     1
2              Ventricular septal defect (HP:0001629) (observed)     1
3                      Right aortic arch (HP:0012020) (observed)     1
4  Abnormal subclavian artery morphology (HP:0031251) (observed)     1
5                   Aortic regurgitation (HP:0001659) (observed)     1
6                   Mitral regurgitation (HP:0001653) (observed)     1

In [28]:
ugKidney_d = { #'No': 'PLACEHOLDER',
 'Right hydrocele': 'Hydrocele testis',
 'Cryptorchidism': 'Cryptorchidism',
 'hypospadia': 'Hypospadias',
 'Bilateral cryptorchidism': 'Bilateral cryptorchidism',
 'Yes': 'Abnormality of the genitourinary system',
 'Balanopreputial hypospadias': 'Hypospadias'}
ugKidneyMapper = OptionColumnMapper(column_name='Urogenital / kidney malformations',
                                                           concept_recognizer=hpo_cr, option_d=ugKidney_d)
column_mapper_list.append(ugKidneyMapper)
ugKidneyMapper.preview_column(dft)

mapping count
0                         Hydrocele testis (HP:0000034) (observed)     1
1                           Cryptorchidism (HP:0000028) (observed)     1
2                              Hypospadias (HP:0000047) (observed)     2
3                 Bilateral cryptorchidism (HP:0008689) (observed)     1
4  Abnormality of the genitourinary system (HP:0000119) (observed)     1

In [29]:
abdominal_d = {
 'Hernia': 'Hernia',
 'diastasis': 'Diastasis recti'}
abdominalMapper = OptionColumnMapper(column_name='Other abdominal',concept_recognizer=hpo_cr, option_d=abdominal_d)
column_mapper_list.append(abdominalMapper)
abdominalMapper.preview_column(dft)

mapping count
0           Hernia (HP:0100790) (observed)     1
1  Diastasis recti (HP:0001540) (observed)     1

In [30]:
pectus_d = { 'Pectus excavatum': 'Pectus excavatum',
 'Pectus carinatum': 'Pectus carinatum'}
excluded = {'No': 'Abnormal sternum morphology'}
pectusMapper = OptionColumnMapper(column_name='Pectus deformity',concept_recognizer=hpo_cr, option_d=pectus_d, excluded_d=excluded)
column_mapper_list.append(pectusMapper)
pectusMapper.preview_column(dft)

mapping count
0  Abnormal sternum morphology (HP:0000766) (excluded)    12
1             Pectus excavatum (HP:0000767) (observed)     2
2             Pectus carinatum (HP:0000768) (observed)     1

In [31]:
other_physical_d = {'Genu valum': 'Genu valgum',
 'Marfanoid habitus with arachnodactyly and elbow flessum': ['Disproportionate tall stature', 'Arachnodactyly'],
 'Brachymetatarsi a': 'Short metatarsal',
 'fallen arches': 'Pes planus',
 'L1 vertebral hypoplasia, with associated kyphosis': ['Vertebral hypoplasia','Kyphosis'],
 'Asymmetric chest': 'Asymmetry of the thorax',
 'Marfan oid habitus with dolichostenomelia': 'Disproportionate tall stature',
 'arachnodactyly': 'Arachnodactyly',
 'flat feet, and elbow flessum, Equinovarus, Postaxial hexadactyly on left foot, Onydhodystrophy': ['Pes planus', 'Limited elbow movement', 'Equinovarus deformity', 'Postaxial foot polydactyly', 'Congenital onychodystrophy'], 
 'Marfanoid habitus, with arachnodactyly, Genu valgum, Metatarsus varus': ['Disproportionate tall stature', 'Genu valgum', 'Metatarsus varus'],
 'Genu valgum': 'Genu valgum'
}
other_physicalMapper = OptionColumnMapper(column_name='Other physical',concept_recognizer=hpo_cr, option_d=other_physical_d)
column_mapper_list.append(other_physicalMapper)
other_physicalMapper.preview_column(dft)

mapping count
0                    Genu valgum (HP:0002857) (observed)     2
1            Talipes equinovarus (HP:0001762) (observed)     1
2  Disproportionate tall stature (HP:0001519) (observed)     3
3                 Arachnodactyly (HP:0001166) (observed)     3
4               Short metatarsal (HP:0010743) (observed)     1
5                     Pes planus (HP:0001763) (observed)     2
6           Vertebral hypoplasia (HP:0008417) (observed)     1
7                       Kyphosis (HP:0002808) (observed)     1
8        Asymmetry of the thorax (HP:0001555) (observed)     1
9            Metatarsus adductus (HP:0001840) (observed)     1

In [32]:
facial_features_d = {'Arched eyebrow': 'Highly arched eyebrow',
 'High andprominentforehead': 'High forehead',
 'Frontal bossing': 'Frontal bossing',
 'Asymmetric occipital plagiocephaly': 'Posterior plagiocephaly',
 'Triangular face': 'Triangular face',
 'Low anterior hairline': 'Low anterior hairline',
 'Prominent eyes': 'Proptosis',
 'Small ears': 'Microtia',
 'low-set ears': 'Low-set ears',
 'Wide nasal bridge': 'Wide nasal bridge',
 'Short philtrum High - archedpalate Downturned of corners ofmouth Prominent chin': 'Mandibular prognathia',
 'Thin upslanted eyebrows': 'Thin eyebrow',
 'Upslanted palpebral fissures': 'Upslanted palpebral fissure',
 'Deeply set eyes': 'Deeply set eye',
 'Buccal hypotonia': 'Hypotonia',
 'High-arched palate': 'High palate',
 'Prominent chin': 'Mandibular prognathia',
 'epicanthus': 'Epicanthus',
 'Low-set ears': 'Low-set ears',
 'Long philtrum': 'Long philtrum',
 'Downturned of corners of mouth': 'Downturned corners of mouth',
 'Retrognathia': 'Retrognathia',
 'Protuding ears': 'Protruding ear',
 'High arched palate': 'High palate',
 'Malar hypoplasia': 'Malar flattening',
 'Long face': 'Long face',
 'Horizontal eyebrows': 'Horizontal eyebrow',
 'Bilateral epicanthus': 'Epicanthus',
 'Posteriorly rotated ears': 'Posteriorly rotated ears',
 'Hypertelorism': 'Hypertelorism',
 'Mild micrognathia': 'Micrognathia',
 'Smooth philtrum': 'Smooth philtrum',
 'Thin lips': 'Thin vermilion border',
 'Gingival hyperplasia': 'Gingival overgrowth',
 'Low set ears': 'Low-set ears',
 'Brachycephaly': 'Brachycephaly',
 'Left preauricular pit': 'Preauricular pit',
 'Narrow ear canals': 'Stenosis of the external auditory canal',
 'Thin upper lip vermilion': 'Thin upper lip vermilion',
 'Mildly broad / low posterior hairline': 'Low posterior hairline',
 'Downslanting palpebral fissures': 'Downslanted palpebral fissures',
 'Strabismus': 'Strabismus',
 'Maxillar hypoplasia': 'Hypoplasia of the maxilla',
 'Synophris': 'Synophrys',
 'Highly arched eyebrow': 'Highly arched eyebrow',
 'with bulbous nasal tip': 'Bulbous nose',
 'Thin upper lips': 'Thin upper lip vermilion',
 'Thin upper lip': 'Thin upper lip vermilion',
 'Short chin': 'Short chin',
 'Full cheeks': 'Full cheeks',
 'Epicanthal folds': 'Epicanthus',
 'Telecanthus': 'Telecanthus',
 'Oval face': 'Oval face',
 'Thick lips': 'Thick vermilion border',
 'Pointed chin': 'Pointed chin',
 'Long nose': 'Long nose',
 'and thin nose': 'Narrow nose',
 #'Microretrognathism': 'Micrognathnia',
 'Downslanted palpebral fissures': 'Downslanted palpebral fissures',
 'with facial asymmetry': 'Facial asymmetry',
# 'and tubular nose': 'PLACEHOLDER',
 'and smooth philtrum': 'Smooth philtrum',
 'Depressed nasal bridge': 'Depressed nasal bridge',
 'Midly downturned corners of the mouth': 'Downturned corners of mouth'}
facial_featuresMapper = OptionColumnMapper(column_name='Facial features',concept_recognizer=hpo_cr, option_d=facial_features_d)
column_mapper_list.append(facial_featuresMapper)
facial_featuresMapper.preview_column(dft)

mapping count
0                     Highly arched eyebrow (HP:0002553) (observed)     2
1                             High forehead (HP:0000348) (observed)     1
2                           Frontal bossing (HP:0002007) (observed)     1
3                   Posterior plagiocephaly (HP:0011327) (observed)     1
4                           Triangular face (HP:0000325) (observed)     2
5                     Low anterior hairline (HP:0000294) (observed)     1
6                                 Proptosis (HP:0000520) (observed)     1
7                                  Microtia (HP:0008551) (observed)     1
8                              Low-set ears (HP:0000369) (observed)     3
9                         Wide nasal bridge (HP:0000431) (observed)     2
10                    Mandibular prognathia (HP:0000303) (observed)     2
11                             Thin eyebrow (HP:0045074) (observed)     1
12              Upslanted palpebral fissure (HP:0000582) (observed)     3
13                           Deeply set eye (HP:0000490) (observed)     2
14                                Hypotonia (HP:0001252) (observed)     2
15                              High palate (HP:0000218) (observed)     3
16                               Epicanthus (HP:0000286) (observed)     3
17                            Long philtrum (HP:0000343) (observed)     2
18              Downturned corners of mouth (HP:0002714) (observed)     2
19                             Retrognathia (HP:0000278) (observed)     2
20                           Protruding ear (HP:0000411) (observed)     1
21                         Malar flattening (HP:0000272) (observed)     2
22                                Long face (HP:0000276) (observed)     1
23                       Horizontal eyebrow (HP:0011228) (observed)     1
24                 Posteriorly rotated ears (HP:0000358) (observed)     1
25                            Hypertelorism (HP:0000316) (observed)     2
26                             Micrognathia (HP:0000347) (observed)     1
27                          Smooth philtrum (HP:0000319) (observed)     3
28                    Thin vermilion border (HP:0000233) (observed)     1
29                      Gingival overgrowth (HP:0000212) (observed)     1
30                            Brachycephaly (HP:0000248) (observed)     1
31                         Preauricular pit (HP:0004467) (observed)     1
32  Stenosis of the external auditory canal (HP:0000402) (observed)     1
33                 Thin upper lip vermilion (HP:0000219) (observed)     4
34                   Low posterior hairline (HP:0002162) (observed)     1
35           Downslanted palpebral fissures (HP:0000494) (observed)     3
36                               Strabismus (HP:0000486) (observed)     1
37                Hypoplasia of the maxilla (HP:0000327) (observed)     1
38                                Synophrys (HP:0000664) (observed)     1
39                             Bulbous nose (HP:0000414) (observed)     1
40                               Short chin (HP:0000331) (observed)     1
41                              Full cheeks (HP:0000293) (observed)     1
42                              Telecanthus (HP:0000506) (observed)     1
43                                Oval face (HP:0000300) (observed)     1
44                   Thick vermilion border (HP:0012471) (observed)     1
45                             Pointed chin (HP:0000307) (observed)     1
46                                Long nose (HP:0003189) (observed)     2
47                              Narrow nose (HP:0000460) (observed)     1
48                         Facial asymmetry (HP:0000324) (observed)     1
49                   Depressed nasal bridge (HP:0005280) (observed)     1

In [33]:
cutaneous_abnormalities_d = {'Stretch marks': 'Striae distensae',
 'Truncal and upper limb hyper trichosis, Palmar erythema': ['Thoracic hypertrichosis', 'Generalized hypertrichosis' 'Erythema'],
 'Hirsutism': 'Hirsutism',
}
cutaneous_abnormalitiesMapper = OptionColumnMapper(column_name='Cutaneous abnormalities',
                                                   concept_recognizer=hpo_cr, option_d=cutaneous_abnormalities_d)
column_mapper_list.append(cutaneous_abnormalitiesMapper)
cutaneous_abnormalitiesMapper.preview_column(dft)

mapping count
0  Striae distensae (HP:0001065) (observed)     1
1    Hypertrichosis (HP:0000998) (observed)     2
2          Erythema (HP:0010783) (observed)     1

In [34]:
additional_clinical_d = {'Missing teeth, Endometriosis, Mixed malignant germ cell tumor of the left ovary': ['Abnormal number of permanent teeth', 'Endometriosis', 'Dysgerminoma'],
 'Relative pain in sensitivity': 'Pain insensitivity',
 'Relative pain insensitivity': 'Pain insensitivity',
 #'No': 'PLACEHOLDER',
 'Unexplained episodes of thrombocytosis': 'Thrombocytosis',
 'Sacro - coccygeal dimple': 'Sacral dimple',
 'Severe constipation': 'Constipation',
 'Sleeping problems': 'Sleep abnormality',
 'Sparse hair, 2 hemangiomas': ['Sparse hair', 'Hemangioma'],
 'Moebius syndrome': 'Congenital facial diplegia',
 'Bifid uvula': 'Bifid uvula'}
additional_clinicalMapper = OptionColumnMapper(column_name='Additional clinical features',
                                                        concept_recognizer=hpo_cr, option_d=additional_clinical_d)
column_mapper_list.append(additional_clinicalMapper)
additional_clinicalMapper.preview_column(dft)

mapping count
0  Agenesis of permanent teeth (HP:0006349) (observed)     1
1                Endometriosis (HP:0030127) (observed)     1
2                     Neoplasm (HP:0002664) (observed)     1
3           Pain insensitivity (HP:0007021) (observed)     2
4               Thrombocytosis (HP:0001894) (observed)     1
5                 Constipation (HP:0002019) (observed)     1
6            Sleep abnormality (HP:0002360) (observed)     1
7                  Sparse hair (HP:0008070) (observed)     1
8   Congenital facial diplegia (HP:0007188) (observed)     1
9                  Bifid uvula (HP:0000193) (observed)     1

In [35]:
import re
def decode_age(age):
    year_month_pattern = '(\d+) years and (\d+) month' # note we do not need to match final s
    year_pattern = '(\d+) years'
    month_pattern = '(\d+) months'
    year_month = re.search(year_month_pattern, age)
    if year_month:
        y = year_month.group(1)
        m = year_month.group(2)
        return f"P{y}Y{m}M"
    year = re.search(year_pattern, age)
    if year:
        y = year.group(1)
        return f"P{y}Y"
    month = re.search(month_pattern, age)
    if month:
        m = month.group(1)
        return f"P{m}M"
    # if we get here, then we failed to match - throw an error so we notice
    raise ValueError(f"Could not decode {age}")
    

dft['isoage'] = dft['Age at last examination ( Year + month )'].apply(lambda x: decode_age(x))

In [36]:
ageMapper = AgeColumnMapper.iso8601(column_name='isoage')
#ageMappper.preview_column(dft)

In [37]:
sexMapper = SexColumnMapper(male_symbol='Male', female_symbol='Female', column_name='Sex')
#sexMapper.preview_column(dft['Sex'])

In [38]:
dft['NM_006924.5'] = dft['Transcript ( NM_006924.5 )'].str.replace(" ","")
srsf1_transcript = "NM_006924.5"
srsf1_symbol = "SRSF1"
vman = VariantManager(df=dft, individual_column_name="individual_id", allele_1_column_name='NM_006924.5',
                     gene_symbol=srsf1_symbol, transcript=srsf1_transcript)

In [39]:
var_d = vman.get_variant_d()
print(f"Retrieved {len(var_d)} unique variants from Variant Validator")

Retrieved 13 unique variants from Variant Validator


In [40]:
varMapper = VariantColumnMapper(variant_column_name="NM_006924.5",
                               variant_d=var_d,
                               default_genotype="heterozygous")

In [41]:
encoder = CohortEncoder(df=dft, 
                        hpo_cr=hpo_cr, 
                        column_mapper_list=column_mapper_list, 
                        individual_column_name="index", 
                        age_at_last_encounter_mapper=ageMapper, 
                        sexmapper=sexMapper,
                        variant_mapper=varMapper,
                        metadata=metadata)
dee28 = Disease(disease_id='OMIM:616211', disease_label='Developmental and epileptic encephalopathy 28')
encoder.set_disease(dee28)

individuals = encoder.get_individuals()

In [42]:
cvalidator = CohortValidator(cohort=individuals, ontology=hpo_ontology, min_hpo=1)
qc = QcVisualizer(cohort_validator=cvalidator)
display(HTML(qc.to_summary_html()))

In [43]:
individuals = cvalidator.get_error_free_individual_list()
phenopackets = [i.to_ga4gh_phenopacket(metadata=metadata.to_ga4gh()) for i in individuals]
table = PhenopacketTable(phenopacket_list=phenopackets)
display(HTML(table.to_html()))

In [44]:
Individual.output_individuals_as_phenopackets(individual_list=individuals,metadata=metadata)

We output 15 GA4GH phenopackets to the directory phenopackets
